<div style="text-align: right">Peter Norvig, 12 Feb 2016<br>Revised 22 Aug 2018, Ken Horton <br> Revised 23 Aug 2018, Professor Bradley Warner</div> 

# A Concrete Introduction to Probability (using Python)

In 1814, Pierre-Simon Laplace [wrote](https://en.wikipedia.org/wiki/Classical_definition_of_probability):

>*Probability theory is nothing but common sense reduced to calculation. ... [Probability] is thus simply a fraction whose numerator is the number of favorable cases and whose denominator is the number of all the cases possible ... when nothing leads us to expect that any one of these cases should occur more than any other.*

![Laplace](https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/AduC_197_Laplace_%28P.S.%2C_marquis_de%2C_1749-1827%29.JPG/180px-AduC_197_Laplace_%28P.S.%2C_marquis_de%2C_1749-1827%29.JPG)
<center><a href="https://en.wikipedia.org/wiki/Pierre-Simon_Laplace">Pierre-Simon Laplace</a><br>1814</center>


Laplace nailed it. To untangle a probability problem, all you have to do is define exactly what the cases are, and careful count the favorable and total cases. Let's be clear on our vocabulary words:


- **[Trial](https://en.wikipedia.org/wiki/Experiment_(probability_theory%29):**
  A single occurrence with an outcome that is uncertain until we observe it. 
  <br>*For example, rolling a single die.*
- **[Outcome](https://en.wikipedia.org/wiki/Outcome_(probability%29):**
  A possible result of a trial; one particular state of the world. What Laplace calls a **case.**
  <br>*For example:* `4`.
- **[Sample Space](https://en.wikipedia.org/wiki/Sample_space):**
  The set of all possible outcomes for the trial. 
  <br>*For example,* `{1, 2, 3, 4, 5, 6}`.
- **[Event](https://en.wikipedia.org/wiki/Event_(probability_theory%29):**
  A subset of outcomes that together have some property we are interested in.
  <br>*For example, the event "even die roll" is the set of outcomes* `{2, 4, 6}`. 
- **[Probability](https://en.wikipedia.org/wiki/Probability_theory):**
  As Laplace said, the probability of an event with respect to a sample space is the "number of favorable cases" (outcomes from the sample space that are in the event) divided by the "number of all the cases" in the sample space (assuming "nothing leads us to expect that any one of these cases should occur more than any other"). Since this is a proper fraction, probability will always be a number between 0 (representing an impossible event) and 1 (representing a certain event).
<br>*For example, the probability of an even die roll is 3/6 = 1/2.*

This notebook will explore these concepts in a concrete way using Python code. The code is meant to be succint and explicit, and fast enough to handle sample spaces with millions of outcomes. If you need to handle trillions, you'll want a more efficient implementation. I also have  [another notebook](http://nbviewer.jupyter.org/url/norvig.com/ipython/ProbabilityParadox.ipynb) that covers  paradoxes in Probability Theory. 

# `P` is for Probability

The code below implements Laplace's quote directly: *Probability is thus simply a fraction whose numerator is the number of favorable cases and whose denominator is the number of all the cases possible.*

In [2]:
# Run this code
import datascience as ds
import numpy as np
from fractions import Fraction

def P(event, space): 
    "The probability of an event, given a sample space."
    return Fraction(cases(favorable(event, space)), 
                    cases(space))

favorable = set.intersection # Outcomes that are in the event and in the sample space
cases     = len              # The number of cases is the length, or size, of a set

A set is an unordered collection of unique elements. You can change a list to a set.

In [3]:
temp_list=[1,3,3,4]
temp_list

[1, 3, 3, 4]

In [4]:
set(temp_list)

{1, 3, 4}

Or create one from scratch.

In [5]:
temp_set={1,3,3,4}
temp_set

{1, 3, 4}

 
# Warm-up Problem: Die Roll

What's the probability of rolling an even number with a single six-sided fair die? Mathematicians traditionally use a single capital letter to denote a sample space; I'll use `D` for the die:

In [6]:
D     = {1, 2, 3, 4, 5, 6} # a sample space
even  = {   2,    4,    6} # an event

P(even, D)

Fraction(1, 2)

Good to confirm what we already knew. We can explore some other events:

In [7]:
prime = {2, 3, 5, 7, 11, 13}
odd   = {1, 3, 5, 7, 9, 11, 13}

In [8]:
P(odd, D)

Fraction(1, 2)

We want the probability of an even or prime die roll, $P(even \cup prime)$

In [9]:
P((even | prime), D) # The probability of an even or prime die roll

Fraction(5, 6)

We want the probability of an odd and prime die roll, $P(odd \cap prime)$.

In [10]:
P((odd & prime), D) # The probability of an odd prime die roll

Fraction(1, 3)

The definition of conditional probability is $$
P(A | B) = {P(A \cap B) \over P(B)}
$$

Find the probability that the die roll is odd given that it is prime.

In [11]:
a= P((odd & prime),D)
b= P(prime,D)
a/b

Fraction(2, 3)

The union rule is 
$$P(A \cup B) = P(A) + P(B) - P(A \cap B)$$

Use this to find the probability of even or prime. Do not use the same formula from above.

In [12]:
P(even,D)+P(prime,D)-P((even & prime),D)

Fraction(5, 6)

Mutually exclusive is the events have not outcomes in common. $$P(A \cap B) = 0 $$

Find the probability that the roll of the die is even and odd.

In [13]:
P((even & odd),D)

Fraction(0, 1)

Independence is when the probability of an event is unchanged by the occurrence of another event. Mathematically, we have independence as $$ P(A | B) = P(A)$$  
or 
$$P(A \cap B) =P(A)P(B)$$

The event even is dependent on prime.

In [14]:
P(even,D) == P((even&prime),D)/P(prime,D)

False

Create a new event, greater than 4. Is this event independent of odd? Show as a logic statement.

In [15]:
greater_4={5,6}
P((greater_4 & odd),D)/P(odd,D)==P(greater_4,D)

True

# Card Problems

Consider dealing a hand of five playing cards. An individual card has a rank and  suit, like `'J♥'` for the Jack of Hearts, and a `deck` has 52 cards:

In [16]:
suits = u'♥♠♦♣'
ranks = u'AKQJT98765432'
deck  = [r + s for r in ranks for s in suits]
len(deck)

52

Now I want to define `Hands` as the sample space of all 5-card combinations from `deck`. The function `itertools.combinations` does most of the work; we than concatenate each combination into a space-separated string:


In [17]:
import itertools

def combos(items, n):
    "All combinations of n items; each combo as a space-separated str."
    return set(map(' '.join, itertools.combinations(items, n)))

Hands = combos(deck, 5)
len(Hands)

2598960

There are too many hands to look at them all, but we can sample:

In [18]:
import random
random.sample(Hands, 7)

['J♦ T♣ 9♦ 8♠ 2♦',
 'Q♥ Q♠ J♣ 5♠ 4♠',
 'A♠ Q♠ J♠ 3♥ 3♣',
 'A♠ K♦ Q♥ 9♥ 7♦',
 'K♠ T♦ 8♣ 4♥ 2♦',
 'K♠ Q♣ 8♦ 7♦ 6♦',
 'Q♠ Q♦ 9♦ 7♣ 2♣']

In [19]:
random.sample(deck, 7)

['8♠', '7♠', '6♥', 'A♥', 'K♦', 'A♣', '5♦']

Now we can answer questions like the probability of being dealt a flush (5 cards of the same suit):

In [20]:
flush = {hand for hand in Hands if any(hand.count(suit) == 5 for suit in suits)}

P(flush, Hands)

Fraction(33, 16660)

Or the probability of four of a kind:

In [21]:
four_kind = {hand for hand in Hands if any(hand.count(rank) == 4 for rank in ranks)}

P(four_kind, Hands)

Fraction(1, 4165)

Find the probability of a full house.

In [22]:
fh = {hand for hand in Hands if any(hand.count(rank) == 3 for rank in ranks)&any(hand.count(rank) == 2 for rank in ranks)}
P(fh, Hands)

Fraction(6, 4165)



# Urn Problems

Around 1700, Jacob Bernoulli wrote about removing colored balls from an urn in his landmark treatise *[Ars Conjectandi](https://en.wikipedia.org/wiki/Ars_Conjectandi)*, and ever since then, explanations of probability have relied on [urn problems](https://www.google.com/search?q=probability+ball+urn). (You'd think the urns would be empty by now.) 

![Jacob Bernoulli](http://www2.stetson.edu/~efriedma/periodictable/jpg/Bernoulli-Jacob.jpg)
<center><a href="https://en.wikipedia.org/wiki/Jacob_Bernoulli">Jacob Bernoulli</a><br>1700</center>

For example, here is a three-part problem [adapted](http://mathforum.org/library/drmath/view/69151.html)  from mathforum.org:

> *An urn contains 6 blue, 9 red, and 8 white balls.  We select six balls at random. What is the probability of each of these  outcomes:*

> - *All balls are red*.
- *3 are blue, and 1 is red, and 2 are white, *.
- *Exactly 4 balls are white*.

We'll start by defining the contents of the urn. A `set` can't contain multiple objects that are equal to each other, so I'll call the blue balls `'B1'` through `'B6'`, rather than trying to have 6 balls all called `'B'`:

In [23]:
def balls(color, n):
    "A set of n numbered balls of the given color."
    return {color + str(i)
            for i in range(1, n + 1)}

urn = balls('B', 6) | balls('R', 9) | balls('W', 8)
urn

{'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'R1',
 'R2',
 'R3',
 'R4',
 'R5',
 'R6',
 'R7',
 'R8',
 'R9',
 'W1',
 'W2',
 'W3',
 'W4',
 'W5',
 'W6',
 'W7',
 'W8'}

Now we can define the sample space, `U6`, as the set of all 6-ball combinations:  

In [24]:
U6 = combos(urn, 6)

random.sample(U6, 5)

['W2 W4 R3 R6 R5 W7',
 'R6 B5 R5 R1 R9 W7',
 'R2 W8 R8 B5 W1 W7',
 'B3 R7 B5 R4 B4 R1',
 'W2 B3 R4 W1 R1 R9']

How many elements in `U6`?

In [25]:
(len(U6))

100947

Define  `select` such that `select('R', 6)` is the event of picking 6 red balls from the urn:

In [26]:
def select(color, n, space=U6):
    "The subset of the sample space with exactly `n` balls of given `color`."
    return {s for s in space if s.count(color) == n}

Now I can answer the six questions:  
1) Probability of selecting 6 red balls.  
2) Probability of selecting 3 blue, 1 red, and 2 white balls.  
3) Probability of selecting 4 white balls.  
4) Probability of selecting 3 red or 3 blue balls.  
5) Probability of selecting 3 white balls given that you selected 3 red balls.  
6) Probability of selecting 3 red balls given that you selected 3 white balls. 

In [27]:
P(select('R', 6), U6) # Probability of selecting 6 red balls.

Fraction(4, 4807)

In [28]:
P(select('B', 3)  & select('R', 1) & select('W', 2), U6) # Probability of selecting 3 blue, 1 red, and 2 white balls.

Fraction(240, 4807)

In [29]:
P(select('W', 4), U6) # Probability of selecting 4 white balls.

Fraction(350, 4807)

In [30]:
P(select('R',3),U6)+P(select('B',3), U6)

Fraction(4016, 9177)

In [31]:
P(select('W', 3) & select("R",3), U6) /P(select("R",3),U6) # Probability of selecting 3 white balls given that you selected 3 red balls.

Fraction(2, 13)

In [32]:
P(select('W', 3) & select("R",3), U6) /P(select("W",3),U6) # Probability of selecting 3 red balls given that you selected 3 white balls.

Fraction(12, 65)

# Counting problems via arithmetic

So far, we have calculated probabilities by "brute force" (using python to exhaustively count all possible outcomes). In the absence of a computer, we can verify these calculations using basic arithmetic. 

## Multiplication Rule  

If I can do the first thing in n ways and the second in m, then there are a total of nm ways. If I have 3 shirts and 5 pants, then there are 15 different arrangements of shirts and pants.

## Combinations
If I select 6 balls from the aforementioned urn, how many ways can I choose 6 out of the 9 red balls? I could select any of the 9 for the first ball, any of 8 remaining for the second, and so on down to any of the remaining 4 for the sixth and final ball. But we don't care about the *order* of the six balls, so divide that product by the number of ways to organize 6 things, which is 6!, giving us 
9 &times; 8 &times; 7 &times; 6 &times; 5 &times; 4 / 6! = 84. 

This is known as a **combination**. In most contexts, this is written as:
$$
{n \choose c}=\frac{n!}{c!(n-c)!}
$$

and is read as $n$ choose $c$, or the number of ways to select $c$ items from $n$ available, without replacement and where order doesn't matter. By definition 0! = 1.

We can translate that to code:

In [33]:
from math import factorial

def choose(n, c):
    "Number of ways to choose c items from a list of n items."
    return factorial(n) // (factorial(n - c) * factorial(c))

In [34]:
factorial(0)

1

In [35]:
choose(9, 6)

84

Now we can verify the answers to the six problems. (Since `P` computes a ratio and `choose` computes a count,
I multiply the left-hand-side by `N`, the length of the sample space, to make both sides be counts.)

In [36]:
N = len(U6)

N * P(select('R', 6), U6) == choose(9, 6)

True

Now we will make use of the multiplication rule with the combinations. By the way, combinations are just a special case of the multiplication rule.  

In [37]:
N * P(select('B', 3) & select('W', 2) & select('R', 1), U6) == choose(6, 3) * choose(8, 2) * choose(9, 1)

True

In [38]:
N * P(select('W', 4), U6) == choose(8, 4) * choose(6 + 9, 2)  # (6 + 9 non-white balls)

True

Complete the fourth problem of the six.

In [39]:
a= N* (P(select('R',3),U6)+P(select('B',3), U6))
b= choose(6,3)*choose(9+8,3)+ choose(9,3)*choose(6+8,3)
a==b

True

For the last two, let's use probability. For the first we have:

In [40]:
P(select('W', 3) & select("R",3), U6) /P(select("R",3),U6)

Fraction(2, 13)

In [41]:
2/13

0.15384615384615385

In [42]:
(choose(8,3) * choose(9,3))/(choose(9,3)*choose(8+6,3))# Probability of selecting 3 white balls given that you selected 3 red balls.

0.15384615384615385

Complete the sixth problem.

In [43]:
a= P(select('W', 3) & select("R",3), U6) /P(select("W",3),U6)
b=(choose(8,3) * choose(9,3))/(choose(8,3)*choose(9+6,3))
print(b)
print(a)

0.18461538461538463
12/65


In [44]:
12/65

0.18461538461538463

We can solve all these problems just by counting; all you ever needed to know about probability problems you learned from Sesame Street:

![The Count](http://img2.oncoloring.com/count-dracula-number-thir_518b77b54ba6c-p.gif)
<center><a href="https://en.wikipedia.org/wiki/Count_von_Count">The Count</a><br>1972&mdash;</center>

## Permutations

Suppose instead that order did in fact matter. This is a permutation. Recall in the urn example above, we divided by the number of ways to arrange 6 things ($6!$). This is because order did not matter and all $6!$ ways represented the same outcome. In a permutation, all $6!$ results would be different outcomes because order matters. A **permutation** is sometimes written as ${}_{n}P_{c}$ and is given by:
$$
\frac{n!}{(n-c)!}
$$

In [45]:
def permute(n, c):
    "Number of ways to select c items from a list of n items when order matters."
    return factorial(n) // (factorial(n - c))

Example: Suppose there are 35 operations research majors in a certain class at USAFA. Three of them will get awards (top GPA, best research and most popular). No cadet can receive more than one award. How many possible ways are there to get the award winners? 

In [46]:
permute(35,3)

39270

# Random Variables

A **random variable** is defined as any mapping from the sample space of an experiment to the real numbers, $\mathbb{R}$.

Example: In the urn example above, you select 6 balls at random from an urn containing blue, red and white balls. Let $X$ be the number of red balls contained in the 6 you selected. $X$ is a random variable because it takes the results of an experiment (list of 6 sampled balls) and maps to a real number (number of red balls selected). Note that there are many possible random variables that could apply to this experiment. 

The domain of a random variable is the list or range of values a random variable can take. For example, the domain of $X$ is $\{0,1,2,3,4,5,6\}$. 

Random variables are often divided into two categories: **discrete** and **continuous**. $X$ is an example of a discrete random variable, since it only takes one of a countable range of values. 

## Discrete random variables

Each discrete random variable also has a **probability mass function**, sometimes called a probability distribution. Often represented as $f_X(x)$, the probability distribution gives the probability that a random variable $X$ takes on an inputted value $x$. For example, 

$$
f_X(0) = P(X=0) = P(\mbox{0 red balls selected}) = \frac{{9\choose 0}{14 \choose 6}}{23\choose 6}
$$


In [47]:
choose(9,0)*choose(14,6)/choose(23,6)

0.029748283752860413

Complete the remaining 6 cases and make a Table of the result. The first column is the number of red balls and the second column is the probability. Note that the sum of all the probabilities must equal 1.

In [57]:
import numpy as np
from datascience import *
nums=[0,1,2,3,4,5,6]
probs= [choose(9,0)*choose(14,6)/choose(23,6),
choose(9,1)*choose(14,5)/choose(23,6),
choose(9,2)*choose(14,4)/choose(23,6),
choose(9,3)*choose(14,3)/choose(23,6),
choose(9,4)*choose(14,2)/choose(23,6),
choose(9,5)*choose(14,1)/choose(23,6),
choose(9,6)*choose(14,0)/choose(23,6)]


reds = Table().with_columns(
    "Number of Red balls", nums,
    "Probability", probs)
print(sum(probs))
reds


1.0000000000000002


Number of Red balls,Probability
0,0.0297483
1,0.17849
2,0.356979
3,0.302892
4,0.113584
5,0.0174745
6,0.00083212


## Continuous random variables

Each continuous random variable has a **probability density function** (pdf). This too is represented by $f_X(x)$, but the function does not return a probability but rather a density. Suppose $Y$ is a continuous random variable that can take any value in the interval $[0,1]$. It has the pdf $f_Y(y) = 2y$ in this interval. Because $Y$ is a continuous random variable the probablility at any specific value is technically 0. We could find the probability that $Y$ takes a value in any range by integrating across this function. For example: 
$$
P(0\leq Y \leq 0.5) = \int_0^{0.5} 2y dy = y^2\bigg|_0^{0.5} = 0.25
$$

## Cumulative distribution functions (CDF)

The **cumulative distribution function** (cdf) is typically given by $F_X(x)$ and applies equivalently for discrete and continuous random variables. This function returns the probability that random variable $X$ takes a value less than or equal to the inputted value. For example, in our urn example, 
$$
F_X(2)=P(X\leq 2) = P(\mbox{we select no more than 2 red balls in our sample})=P(X=0)+P(X=1)+P(X=2)
$$

Considering our continuous random variable $Y$:
$$
F_Y(y)=P(Y\leq y) = \int_0^y 2zdz = z^2\bigg|_0^y = y^2
$$